In [1]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_addons as tfa
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
print(tf.__version__)
AUTOTUNE = tf.data.experimental.AUTOTUNE

import deepcell
from deepcell_tracking.utils import Track, load_trks
from sklearn.model_selection import train_test_split
from deepcell.utils.data_utils import reshape_movie
from deepcell.utils.transform_utils import erode_edges
from deepcell.data import split_dataset
from deepcell_toolbox.processing import normalize, histogram_normalization

import spektral

2.5.0


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [2]:
%%time
path_temp_trks = '/data/3T3_nuc_s0-s2.trks'
all_data = [load_trks(path_temp_trks)]

CPU times: user 442 ms, sys: 864 ms, total: 1.31 s
Wall time: 1.31 s


In [4]:
len(all_data)

1

In [5]:
len(all_data[0])

3

In [7]:
all_data[0]

{'lineages': [{1: {'label': 1,
    'frames': [0,
     1,
     2,
     3,
     4,
     5,
     6,
     7,
     8,
     9,
     10,
     11,
     12,
     13,
     14,
     15,
     16,
     17,
     18,
     19,
     20,
     21,
     22,
     23,
     24,
     25,
     26,
     27,
     28,
     29,
     30,
     31,
     32,
     33,
     34],
    'frame_text': '[0-34]',
    'capped': True,
    'cap_type': 'division',
    'frame_div': 35,
    'death': False,
    'death_type': None,
    'frame_died': None,
    'frame_merged': None,
    'parent': None,
    'daughters': [108, 109],
    'merged_from': [],
    'merges_into': None,
    'overlaps': {'debris_partial': [], 'debris_total': []}},
   2: {'label': 2,
    'frames': [0,
     1,
     2,
     3,
     4,
     5,
     6,
     7,
     8,
     9,
     10,
     11,
     12,
     13,
     14,
     15,
     16,
     17,
     18,
     19,
     20,
     21,
     22,
     23,
     24,
     25,
     26,
     27,
     28,
     29,
     30,
     3

In [8]:
%%time
all_tracks = [Track(tracked_data=d) for d in all_data]

CPU times: user 1min 50s, sys: 5.18 s, total: 1min 56s
Wall time: 1min 46s


In [9]:
len(all_tracks)

1

In [12]:
%%time
from deepcell_tracking.utils import concat_tracks
track_info = concat_tracks(all_tracks)

CPU times: user 267 ms, sys: 376 ms, total: 643 ms
Wall time: 642 ms


In [13]:
type(track_info)

dict

In [19]:
all_tracks[0].lineages[0]

{1: {'label': 1,
  'parent': None,
  'daughters': [108, 109],
  'frames': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34]},
 2: {'label': 2,
  'parent': None,
  'daughters': [],
  'frames': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70]},
 3: {'label': 3,
  'parent': None,
  'daughters': [],
  'frames': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,


In [22]:
print(track_info.get('lineages'))

None


In [23]:
type(track_info.get('lineages'))

NoneType

In [24]:
for k, v in track_info.items():
    print(k, v.shape)

appearances (11, 71, 168, 32, 32, 1)
centroids (11, 71, 168, 2)
morphologies (11, 71, 168, 3)
adj_matrices (11, 71, 168, 168)
norm_adj_matrices (11, 71, 168, 168)
temporal_adj_matrices (11, 70, 168, 168, 3)


In [25]:
type(track_info.items())

dict_items

In [28]:
track_info['appearances']

array([[[[[[ 813.0156 ],
           [1018.9375 ],
           [1223.5312 ],
           ...,
           [1554.2188 ],
           [1107.875  ],
           [ 784.84375]],

          [[ 797.15625],
           [ 989.2063 ],
           [1200.354  ],
           ...,
           [1557.3821 ],
           [1102.4268 ],
           [ 778.49316]],

          [[ 738.25   ],
           [ 878.7761 ],
           [1114.2671 ],
           ...,
           [1569.1316 ],
           [1082.1904 ],
           [ 754.9053 ]],

          ...,

          [[ 507.99292],
           [ 536.90063],
           [ 552.49316],
           ...,
           [ 604.48   ],
           [ 596.27905],
           [ 588.86523]],

          [[ 499.84888],
           [ 520.99976],
           [ 557.95215],
           ...,
           [ 591.188  ],
           [ 575.82056],
           [ 586.2754 ]],

          [[ 497.65625],
           [ 516.71875],
           [ 559.4219 ],
           ...,
           [ 587.6094 ],
           [ 570.3125 ],
   

In [29]:
appearances = track_info['appearances']
    
app_shape = np.shape(appearances)

In [30]:
app_shape

(11, 71, 168, 32, 32, 1)

In [31]:
appearances = np.swapaxes(appearances, 1,2) #(0,2,1,3,4,5))

In [32]:
appearances.shape

(11, 168, 71, 32, 32, 1)

In [33]:
appearances = np.reshape(appearances, [-1, app_shape[1], app_shape[3], app_shape[4], app_shape[5]])

In [34]:
track_info['appearances'].shape

(11, 71, 168, 32, 32, 1)

In [35]:
appearances.shape

(1848, 71, 32, 32, 1)

In [36]:
app_shape

(11, 71, 168, 32, 32, 1)

In [37]:
input_dict = {'appearances': appearances}
    
output_dict = {'appearances_decoder': appearances}
    
dataset = tf.data.Dataset.from_tensor_slices((input_dict, output_dict))

In [38]:
type(dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [39]:
dataset

<TensorSliceDataset shapes: ({appearances: (71, 32, 32, 1)}, {appearances_decoder: (71, 32, 32, 1)}), types: ({appearances: tf.float32}, {appearances_decoder: tf.float32})>

In [41]:
dataset['appearances']

TypeError: 'TensorSliceDataset' object is not subscriptable